In [1]:
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from lightgbm.sklearn import LGBMRegressor
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc

%matplotlib inline


df = pd.read_csv('/Users/yangdongjae/Desktop/2020/대외활동/2020년 공공 빅데이터 청년 인턴십/실무형 프로젝트/roundabout_project/data/2017청주 교차로 리얼 최종 0813목.csv', encoding = 'cp949')

/Users/yangdongjae/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
df.columns

Index(['교차로명', '통행량8', '통행량13', '통행량18', '좌회전 비율8', '좌회전 비율13', '좌회전 비율18',
       '위도', '경도', '사고건수', '사망 건수', '중상 건 수', '경상 건 수', '부상 건 수', '사망자 수',
       '중상자 수', '경상자 수', '부상자 수', '교차로 반경 50m 내 무인교통단속카메라 수',
       '교차로 반경 50m 내 신호등 수', '교차로 반경 50m 내 신호등 유무', 'LCLAS_경상사고', 'LCLAS_중상사고',
       'LCLAS_사망사고', 'LCLAS_부상신고사고', 'SCLAS_차대차', 'SCLAS_차량단독', 'SCLAS_차대사람',
       'DRNKG', 'KID', 'ODSN', 'WLKG', 'VIOLT_CN_안전거리미확보',
       'VIOLT_CN_안전운전의무불이행', 'VIOLT_CN_기타', 'VIOLT_CN_신호위반',
       'VIOLT_CN_보행자보호의무위반', 'VIOLT_CN_교차로통행방법위반', 'VIOLT_CN_중앙선침범',
       'VIOLT_CN_과속'],
      dtype='object')

In [3]:
df.set_index(['교차로명'])

,통행량8,통행량13,통행량18,좌회전 비율8,좌회전 비율13,좌회전 비율18,위도,경도,사고건수,사망 건수,...,ODSN,WLKG,VIOLT_CN_안전거리미확보,VIOLT_CN_안전운전의무불이행,VIOLT_CN_기타,VIOLT_CN_신호위반,VIOLT_CN_보행자보호의무위반,VIOLT_CN_교차로통행방법위반,VIOLT_CN_중앙선침범,VIOLT_CN_과속
교차로명,,,,,,,,,,,,,,,,,,,,,
흥덕4,4401,3121,4571,17.660000,18.260000,20.060000,36.645295,127.478596,8,0,...,2,1,3,6,0,3,0,0,0,0
현도면사무소3,2427,1589,2269,18.294190,13.341724,7.051565,36.486378,127.423642,3,0,...,2,1,1,2,1,0,0,0,0,0
효촌분기점3,3331,2314,3107,32.332633,25.972342,34.019955,36.585801,127.508794,0,0,...,1,0,0,1,0,0,0,0,0,0
터미널4,6965,5178,7569,7.365398,6.701429,6.856916,36.628709,127.428424,10,0,...,1,0,0,2,1,8,0,0,0,0
하이닉스반도체4,4821,3490,4967,18.630000,19.710000,18.360000,36.653656,127.435323,6,0,...,1,1,0,5,1,3,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
석판분기점4,1701,977,1792,28.810000,25.590000,39.400000,36.602800,127.440370,1,0,...,0,0,0,1,0,0,0,0,0,0
서문교하상도로3,1738,770,1991,19.332566,28.961039,30.989453,36.636148,127.484193,3,0,...,2,1,0,7,1,2,0,0,0,0
서청주교4,6176,4882,6546,30.930000,25.170000,23.480000,36.646470,127.422813,7,0,...,1,0,0,6,1,0,0,3,0,0


In [4]:
scaler = MinMaxScaler()

df['통행량8'] = scaler.fit_transform(df['통행량8'].values.reshape(-1,1))
df['통행량13'] = scaler.fit_transform(df['통행량13'].values.reshape(-1,1))
df['통행량18'] = scaler.fit_transform(df['통행량18'].values.reshape(-1,1))
df['LCLAS_경상사고'] = scaler.fit_transform(df['LCLAS_경상사고'].values.reshape(-1,1))
df['LCLAS_중상사고'] = scaler.fit_transform(df['LCLAS_중상사고'].values.reshape(-1,1))
df['LCLAS_사망사고'] = scaler.fit_transform(df['LCLAS_사망사고'].values.reshape(-1,1))
df['LCLAS_부상신고사고'] = scaler.fit_transform(df['LCLAS_부상신고사고'].values.reshape(-1,1))
df['사망 건수'] = scaler.fit_transform(df['사망 건수'].values.reshape(-1,1))
df['중상 건 수'] = scaler.fit_transform(df['중상 건 수'].values.reshape(-1,1))
df['경상 건 수'] = scaler.fit_transform(df['경상 건 수'].values.reshape(-1,1))
df['VIOLT_CN_안전거리미확보'] = scaler.fit_transform(df['VIOLT_CN_안전거리미확보'].values.reshape(-1,1))
df['VIOLT_CN_안전운전의무불이행'] = scaler.fit_transform(df['VIOLT_CN_안전거리미확보'].values.reshape(-1,1))
df['VIOLT_CN_기타'] = scaler.fit_transform(df['VIOLT_CN_안전거리미확보'].values.reshape(-1,1))
df['VIOLT_CN_신호위반'] = scaler.fit_transform(df['VIOLT_CN_안전거리미확보'].values.reshape(-1,1))
df['VIOLT_CN_보행자보호의무위반 '] = scaler.fit_transform(df['VIOLT_CN_안전거리미확보'].values.reshape(-1,1))
df['VIOLT_CN_교차로통행방법위반'] = scaler.fit_transform(df['VIOLT_CN_안전거리미확보'].values.reshape(-1,1))
df['VIOLT_CN_중앙선침범'] = scaler.fit_transform(df['VIOLT_CN_안전거리미확보'].values.reshape(-1,1))
df['VIOLT_CN_과속 '] = scaler.fit_transform(df['VIOLT_CN_안전거리미확보'].values.reshape(-1,1))

In [5]:
df.columns

Index(['교차로명', '통행량8', '통행량13', '통행량18', '좌회전 비율8', '좌회전 비율13', '좌회전 비율18',
       '위도', '경도', '사고건수', '사망 건수', '중상 건 수', '경상 건 수', '부상 건 수', '사망자 수',
       '중상자 수', '경상자 수', '부상자 수', '교차로 반경 50m 내 무인교통단속카메라 수',
       '교차로 반경 50m 내 신호등 수', '교차로 반경 50m 내 신호등 유무', 'LCLAS_경상사고', 'LCLAS_중상사고',
       'LCLAS_사망사고', 'LCLAS_부상신고사고', 'SCLAS_차대차', 'SCLAS_차량단독', 'SCLAS_차대사람',
       'DRNKG', 'KID', 'ODSN', 'WLKG', 'VIOLT_CN_안전거리미확보',
       'VIOLT_CN_안전운전의무불이행', 'VIOLT_CN_기타', 'VIOLT_CN_신호위반',
       'VIOLT_CN_보행자보호의무위반', 'VIOLT_CN_교차로통행방법위반', 'VIOLT_CN_중앙선침범',
       'VIOLT_CN_과속', 'VIOLT_CN_보행자보호의무위반 ', 'VIOLT_CN_과속 '],
      dtype='object')

In [6]:
df_y = df['사고건수']

In [7]:
df['LCLAS_중상사고'].value_counts

<bound method IndexOpsMixin.value_counts of 0     0.285714
1     0.285714
2     0.000000
3     0.285714
4     0.000000
        ...   
65    0.000000
66    0.714286
67    0.000000
68    0.142857
69    0.428571
Name: LCLAS_중상사고, Length: 70, dtype: float64>

In [8]:
df_y

0      8
1      3
2      0
3     10
4      6
      ..
65     1
66     3
67     7
68    11
69     4
Name: 사고건수, Length: 70, dtype: int64

In [9]:
df.drop(columns = ['교차로명','위도','경도','부상 건 수', '사망 건수','중상 건 수', '경상 건 수'],inplace = True)

In [10]:
df.columns

Index(['통행량8', '통행량13', '통행량18', '좌회전 비율8', '좌회전 비율13', '좌회전 비율18', '사고건수',
       '사망자 수', '중상자 수', '경상자 수', '부상자 수', '교차로 반경 50m 내 무인교통단속카메라 수',
       '교차로 반경 50m 내 신호등 수', '교차로 반경 50m 내 신호등 유무', 'LCLAS_경상사고', 'LCLAS_중상사고',
       'LCLAS_사망사고', 'LCLAS_부상신고사고', 'SCLAS_차대차', 'SCLAS_차량단독', 'SCLAS_차대사람',
       'DRNKG', 'KID', 'ODSN', 'WLKG', 'VIOLT_CN_안전거리미확보',
       'VIOLT_CN_안전운전의무불이행', 'VIOLT_CN_기타', 'VIOLT_CN_신호위반',
       'VIOLT_CN_보행자보호의무위반', 'VIOLT_CN_교차로통행방법위반', 'VIOLT_CN_중앙선침범',
       'VIOLT_CN_과속', 'VIOLT_CN_보행자보호의무위반 ', 'VIOLT_CN_과속 '],
      dtype='object')

In [11]:
df.head()

,통행량8,통행량13,통행량18,좌회전 비율8,좌회전 비율13,좌회전 비율18,사고건수,사망자 수,중상자 수,경상자 수,...,VIOLT_CN_안전거리미확보,VIOLT_CN_안전운전의무불이행,VIOLT_CN_기타,VIOLT_CN_신호위반,VIOLT_CN_보행자보호의무위반,VIOLT_CN_교차로통행방법위반,VIOLT_CN_중앙선침범,VIOLT_CN_과속,VIOLT_CN_보행자보호의무위반,VIOLT_CN_과속
0,0.500880,0.533348,0.546719,17.660000,18.260000,20.060000,8,0,3,9,...,1.000000,1.000000,1.000000,1.000000,0,1.000000,1.000000,0,1.000000,1.000000
1,0.211268,0.185799,0.198669,18.294190,13.341724,7.051565,3,0,2,1,...,0.333333,0.333333,0.333333,0.333333,0,0.333333,0.333333,0,0.333333,0.333333
2,0.343897,0.350272,0.325370,32.332633,25.972342,34.019955,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000
3,0.877054,1.000000,1.000000,7.365398,6.701429,6.856916,10,0,2,16,...,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000
4,0.562500,0.617060,0.606592,18.630000,19.710000,18.360000,6,0,0,10,...,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   통행량8                      70 non-null     float64
 1   통행량13                     70 non-null     float64
 2   통행량18                     70 non-null     float64
 3   좌회전 비율8                   70 non-null     float64
 4   좌회전 비율13                  70 non-null     float64
 5   좌회전 비율18                  70 non-null     float64
 6   사고건수                      70 non-null     int64  
 7   사망자 수                     70 non-null     int64  
 8   중상자 수                     70 non-null     int64  
 9   경상자 수                     70 non-null     int64  
 10  부상자 수                     70 non-null     int64  
 11  교차로 반경 50m 내 무인교통단속카메라 수  70 non-null     int64  
 12  교차로 반경 50m 내 신호등 수        70 non-null     int64  
 13  교차로 반경 50m 내 신호등 유무       70 non-null     int64  
 14  LCLAS_경상사고  

In [13]:
df.rename(columns = {'사고건수':'y'},inplace = True)

In [14]:
df

,통행량8,통행량13,통행량18,좌회전 비율8,좌회전 비율13,좌회전 비율18,y,사망자 수,중상자 수,경상자 수,...,VIOLT_CN_안전거리미확보,VIOLT_CN_안전운전의무불이행,VIOLT_CN_기타,VIOLT_CN_신호위반,VIOLT_CN_보행자보호의무위반,VIOLT_CN_교차로통행방법위반,VIOLT_CN_중앙선침범,VIOLT_CN_과속,VIOLT_CN_보행자보호의무위반,VIOLT_CN_과속
0,0.500880,0.533348,0.546719,17.660000,18.260000,20.060000,8,0,3,9,...,1.000000,1.000000,1.000000,1.000000,0,1.000000,1.000000,0,1.000000,1.000000
1,0.211268,0.185799,0.198669,18.294190,13.341724,7.051565,3,0,2,1,...,0.333333,0.333333,0.333333,0.333333,0,0.333333,0.333333,0,0.333333,0.333333
2,0.343897,0.350272,0.325370,32.332633,25.972342,34.019955,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000
3,0.877054,1.000000,1.000000,7.365398,6.701429,6.856916,10,0,2,16,...,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000
4,0.562500,0.617060,0.606592,18.630000,19.710000,18.360000,6,0,0,10,...,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.104754,0.046960,0.126550,28.810000,25.590000,39.400000,1,0,0,1,...,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000
66,0.110182,0.000000,0.156637,19.332566,28.961039,30.989453,3,0,0,3,...,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000
67,0.761297,0.932849,0.845328,30.930000,25.170000,23.480000,7,0,0,10,...,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000
68,0.474765,0.586887,0.535984,16.054937,14.000596,19.555556,11,0,1,14,...,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.000000,0,0.000000,0.000000


In [15]:
df.columns

Index(['통행량8', '통행량13', '통행량18', '좌회전 비율8', '좌회전 비율13', '좌회전 비율18', 'y',
       '사망자 수', '중상자 수', '경상자 수', '부상자 수', '교차로 반경 50m 내 무인교통단속카메라 수',
       '교차로 반경 50m 내 신호등 수', '교차로 반경 50m 내 신호등 유무', 'LCLAS_경상사고', 'LCLAS_중상사고',
       'LCLAS_사망사고', 'LCLAS_부상신고사고', 'SCLAS_차대차', 'SCLAS_차량단독', 'SCLAS_차대사람',
       'DRNKG', 'KID', 'ODSN', 'WLKG', 'VIOLT_CN_안전거리미확보',
       'VIOLT_CN_안전운전의무불이행', 'VIOLT_CN_기타', 'VIOLT_CN_신호위반',
       'VIOLT_CN_보행자보호의무위반', 'VIOLT_CN_교차로통행방법위반', 'VIOLT_CN_중앙선침범',
       'VIOLT_CN_과속', 'VIOLT_CN_보행자보호의무위반 ', 'VIOLT_CN_과속 '],
      dtype='object')

In [16]:
x = df.drop(columns = ['y'])


In [17]:
y = df['y']

In [41]:
x['SCLAS_차대차'] , 'SCL

11     3
3     11
1      3
11     3
10     4
      ..
1      3
13     0
10     4
9      1
7      9
Name: SCLAS_차대차, Length: 70, dtype: int64

In [39]:
train_x , test_y , train_x , test_y = train_test_split(x,y, test_size = 0.33, random_state = 42)

In [37]:
from sklearn.model_selection import KFold,cross_val_score
def rmse_cv(model):
    n_fold = 10
    kf = KFold(n_fold, shuffle=True,random_state=42).get_n_splits(train_x)
    rmse = np.sqrt(-cross_val_score(model,train_x.values,train_y,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [36]:
rf = RandomForestRegressor()

rf.fit(train_x, train_y)

/Users/yangdongjae/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

/Users/yangdongjae/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [24]:
print("교차 검증 점수: {}".format(np.mean(rmse_cv(rf))))

/Users/yangdongjae/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/yangdongjae/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:528: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
from sklearn.metrics import mean_squared_error
rf.fit(x_train,y_train)
pred = rf.predict(x_test)
mean_squared_error(y_test,pred)




In [ ]:
np.sqrt(mean_squared_error(y_test,pred))

In [ ]:
def set_font(font_file='malgun.ttf'): # 한글 출력
    font_dir = '/user/sanchez/Library/Fonts'
    font_name = fm.FontProperties(fname=os.path.join(font_dir, font_file)).get_name()
    rc('font', family=font_name)
set_font()

In [ ]:
df

In [ ]:
feat_importances = pd.Series(rf.feature_importances_, index=train_x.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.title('rf featrue importances')
plt.show()